In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import matplotlib 
matplotlib.use('Agg') 
% matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import h5py
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,exception_verbosity=high"
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, SpatialDropout2D
from keras.optimizers import SGD

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


In [4]:
# dimensions of our images.
img_width, img_height = 150, 150
train_data_dir = '../UECFOOD100/data/train'
validation_data_dir = '../UECFOOD100/data/validation'
nb_train_samples = 10183
nb_validation_samples = 4428
nb_epoch = 50
batch_size = 32

# path to the model weights file.
weights_path = 'D:/DSR/neural-networks-and-deep-learning/CatsandDogs/models/vgg16_weights.h5'
top_model_weights_path = './bottleneck_features_11_12_2016.h5'

In [ ]:
# # this is the augmentation configuration we will use for training
# train_augmentations = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

# # this is the augmentation configuration we will use for testing:
# # only rescaling
# test_augmentations = ImageDataGenerator(rescale=1./255)

# train_generator = train_augmentations.flow_from_directory(
#         train_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size,
#         class_mode='categorical')

# validation_generator = test_augmentations.flow_from_directory(
#         validation_data_dir,
#         target_size=(img_width, img_height),
#         batch_size=batch_size,
#         class_mode='categorical')

In [6]:
#Extract bottleneck features usng VGG16:

datagen = ImageDataGenerator(rescale=1./255)

# build the VGG16 network
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

train_bottleneck_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

train_labels = train_bottleneck_generator.classes

bottleneck_features_train = model.predict_generator(train_bottleneck_generator, nb_train_samples)

np.save(open('bottleneck_features_train_12_2016.npy', 'ba'), bottleneck_features_train)

validation_bottleneck_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

validation_labels = validation_bottleneck_generator.classes

bottleneck_features_validation = model.predict_generator(validation_bottleneck_generator, nb_validation_samples)
np.save(open('bottleneck_features_validation_12_2016.npy', 'ba'), bottleneck_features_validation)

Model loaded.
Found 10183 images belonging to 100 classes.
Found 4428 images belonging to 100 classes.


In [9]:
validation_bottleneck_generator.

array([ 0,  0,  0, ..., 99, 99, 99])

In [10]:
train_labels_encoded = np.zeros((len(train_labels), 100), dtype=int)
for index, element in enumerate(train_labels):
    train_labels_encoded[index,element] = 1
validation_labels_encoded = np.zeros((len(validation_labels), 100), dtype=int)
for index, element in enumerate(validation_labels):
    validation_labels_encoded[index,element] = 1

sgd = SGD(lr=0.01, clipnorm=1.)

# bottleneck_features_train = np.load(open('bottleneck_features_train_12_2016.npy','rb'))
# bottleneck_features_validation = np.load(open('bottleneck_features_validation_12_2016.npy', 'rb'))


model = Sequential()
model.add(Flatten(input_shape=bottleneck_features_train.shape[1:]))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(100, activation='sigmoid'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

nb_epoch = 100

model.fit(bottleneck_features_train, train_labels_encoded,
          nb_epoch=nb_epoch, 
          batch_size=32,
          validation_data=(bottleneck_features_validation, validation_labels_encoded))

nb_epoch = 50

##### model.save_weights(top_model_weights_path)
# model.save_weights('./bottleneck_fc_model_11_12_16.h5')

Train on 10183 samples, validate on 4428 samples
Epoch 1/100
10183/10183 [==============================] - 2s - loss: 4.5298 - acc: 0.0407 - val_loss: 4.4191 - val_acc: 0.0598
Epoch 2/100
10183/10183 [==============================] - 2s - loss: 4.3906 - acc: 0.0629 - val_loss: 4.2527 - val_acc: 0.0879
Epoch 3/100
10183/10183 [==============================] - 2s - loss: 4.2393 - acc: 0.0774 - val_loss: 4.0609 - val_acc: 0.1129
Epoch 4/100
10183/10183 [==============================] - 2s - loss: 4.0738 - acc: 0.0902 - val_loss: 3.8605 - val_acc: 0.1163
Epoch 5/100
10183/10183 [==============================] - 2s - loss: 3.9118 - acc: 0.1083 - val_loss: 3.6328 - val_acc: 0.1484
Epoch 6/100
10183/10183 [==============================] - 2s - loss: 3.7436 - acc: 0.1225 - val_loss: 3.4321 - val_acc: 0.1518
Epoch 7/100
10183/10183 [==============================] - 2s - loss: 3.5897 - acc: 0.1528 - val_loss: 3.2712 - val_acc: 0.2103
Epoch 8/100
10183/10183 [==============================

In [5]:
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# load the weights of the VGG16 networks
# (trained on ImageNet, won the ILSVRC competition in 2014)
# note: when there is a complete match between your model definition
# and your weight savefile, you can simply call model.load_weights(filename)
assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')
model.output_shape[1:]

Model loaded.


(512, 4, 4)

In [7]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))

top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(100, activation='sigmoid'))



# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights('./bottleneck_fc_model_11_12_16.h5')

# add the model on top of the convolutional base
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.

# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
# #               optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#               metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=60,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

# fine-tune the model
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Found 10183 images belonging to 100 classes.
Found 4428 images belonging to 100 classes.
Epoch 1/50
10183/10183 [==============================] - 524s - loss: 2.8294 - acc: 0.3113 - val_loss: 1.8002 - val_acc: 0.5300
Epoch 2/50
10183/10183 [==============================] - 525s - loss: 2.5478 - acc: 0.3606 - val_loss: 1.6803 - val_acc: 0.5556
Epoch 3/50
10183/10183 [==============================] - 530s - loss: 2.4265 - acc: 0.3769 - val_loss: 1.6622 - val_acc: 0.5619
Epoch 4/50
10183/10183 [==============================] - 524s - loss: 2.3704 - acc: 0.3899 - val_loss: 1.6379 - val_acc: 0.5693
Epoch 5/50
10183/10183 [==============================] - 522s - loss: 2.2832 - acc: 0.4153 - val_loss: 1.6525 - val_acc: 0.5596
Epoch 6/50
10183/10183 [==============================] - 522s - loss: 2.2652 - acc: 0.4149 - val_loss: 1.6262 - val_acc: 0.5734
Epoch 7/50
10183/10183 [==============================] - 523s - loss: 2.2424 - acc: 0.4235 - val_loss: 1.5962 - val_acc: 0.5725
Epoch 8/

In [8]:
# model.save('fine_tuned_model_all_newRun.h5')